<a href="https://colab.research.google.com/github/Elwing-Chou/ml0804/blob/master/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
z = zipfile.ZipFile("drive/My Drive/additional/train.zip")
z.extractall("transfer")

In [ ]:
import glob
import pandas as pd
dogs = glob.glob("transfer/train/dog.*")
cats = glob.glob("transfer/train/cat.*")
df = pd.DataFrame({
    "path":dogs + cats,
    "ans":[0] * len(dogs) + [1] * len(cats)
})
df

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
mlp = [
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]
for l in vgg.layers:
    l.trainable = False

layers = vgg.layers + mlp
model = Sequential(layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import preprocess_input

img = load_img(df["path"][0], target_size=(224, 224))
p = preprocess_input(np.array(img))
print(p)
plt.imshow(img)